In [155]:
import numpy as np
import pandas as pd

In [156]:
df = pd.read_csv(r"C:\Users\user\Downloads\Bank_Personal_Loan_Modelling.csv")


In [157]:
df= df[['Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard', 'Personal Loan']]

In [158]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

In [159]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [160]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=32)

In [196]:
from keras.models import Sequential
from keras.layers import Dense
from pyswarms.single import GlobalBestPSO

In [197]:
def pso_model(input_dim, output_dim, nodes, n=1, name='model'):
    model = Sequential(name=name)
    for i in range(n):
        model.add(Dense(nodes, input_dim=input_dim, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [198]:
features = X.shape[1]
classes = len(df['Personal Loan'].unique())

In [199]:
n_layers = 2
model = pso_model(features, classes,4, n_layers)
model.summary()

callback = model.fit(X_train, Y_train,batch_size=5,epochs=50,validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 4)                 52        
                                                                 
 dense_13 (Dense)            (None, 4)                 20        
                                                                 
 dense_14 (Dense)            (None, 1)                 5         
                                                                 
Total params: 77
Trainable params: 77
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
700/700 [==============================] - 1s 1ms/step - loss: 0.3431 - accuracy: 0.9049 - val_loss: 0.2347 - val_accuracy: 0.9013
Epoch 2/50
700/700 [==============================] - 1s 1ms/step - loss: 0.2058 - accuracy: 0.9051 - val_loss: 0.1821 - val_accuracy: 0.9013
Epoch 3/50
700/700 [=========================

In [200]:
print('Test accuracy:', score[1])

Test accuracy: 0.972000002861023


In [201]:
def get_shape(model):
    weights_layer = model.get_weights()
    shapes = []
    for weights in weights_layer:
        shapes.append(weights.shape)
    return shapes
def set_shape(weights,shapes):
    new_weights = []
    index=0
    for shape in shapes:
        if(len(shape)>1):
            n_nodes = np.prod(shape)+index
        else:
            n_nodes=shape[0]+index
        tmp = np.array(weights[index:n_nodes]).reshape(shape)
        new_weights.append(tmp)
        index=n_nodes
    return new_weights

In [202]:
def evaluate_nn(W, shape,X_train=X_train, Y_train=Y_train):
    results = []
    for weights in W:
        model.set_weights(set_shape(weights,shape))
        score = model.evaluate(X_train, Y_train, verbose=0)
        results.append(1-score[1])
    return results

In [203]:
shape = get_shape(model)
x_max = 1.0 * np.ones(83)
x_min = -1.0 * x_max
bounds = (x_min, x_max)
options = {'c1': 0.4, 'c2': 0.8, 'w': 0.4}
optimizer = GlobalBestPSO(n_particles=25, dimensions=83,options=options, bounds=bounds)

In [204]:
cost, pos = optimizer.optimize(evaluate_nn, 15, X_train=X_train, Y_train=Y_train,shape=shape)
model.set_weights(set_shape(pos,shape))
score = model.evaluate(X_test, Y_test)

2023-04-05 22:03:17,891 - pyswarms.single.global_best - INFO - Optimize for 15 iters with {'c1': 0.4, 'c2': 0.8, 'w': 0.4}
pyswarms.single.global_best: 100%|█████████████████████████████████████████████████████████████|15/15, best_cost=0.0691
2023-04-05 22:03:59,747 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.06914287805557251, best pos: [-0.03871953  0.29903638 -0.8029468   0.27475548 -0.6606405  -0.56256726
 -0.98505561  0.30565651  0.18793528  0.69020408  0.24923037  0.05021557
 -0.46135808  0.17690588 -0.66632166  0.5553383   0.23453064  0.66847083
 -0.1034632  -0.10857127  0.60207434  0.79120588  0.28913576 -0.16476445
 -0.04208235  0.19722512 -0.97418422 -0.84442241  0.69975047  0.19507467
  0.25772446  0.14667279  0.43148055  0.19902097  0.04501889  0.40937487
  0.67322941  0.3458522  -0.01938289  0.21212475 -0.7207811  -0.32915339
  0.03077296 -0.26227863  0.3072528  -0.69356425 -0.06503643 -0.32956031
  0.68989193 -0.28958538 -0.00889672  0.5927

47/47 [==============================] - 0s 802us/step - loss: 0.5386 - accuracy: 0.9193


In [205]:
print('Test accuracy:', score[1])

Test accuracy: 0.9193333387374878
